In [1]:
import requests
import json
import re
import operator

In [2]:
draft_group_url = 'https://api.draftkings.com/draftgroups/v1/draftgroups/' 
draft_tables_suffix = '/draftables?format=json'
base_url = 'https://www.draftkings.com'
contest_suffix = '/lobby/getcontests'
get_available_players_suffix = '/lineup/getavailableplayers'
BASKETBALL = "?sport=nba"
HOCKEY = "?sport=nhl"

In [13]:
red_eye = False
sport = HOCKEY

In [14]:
def fixPlayerName(name):
    return re.sub('([^\s\w]|_)+', '', name)

In [15]:
response = requests.get(base_url + contest_suffix + sport)

In [16]:
data = json.loads(response.text)

In [17]:
found = False
for contest in data['Contests']:
    if not found:
        dict = contest['pd']
        #print (dict)
        for key in dict:
            val = dict.get('Cash')
            try:
                if float(val) == 4.5 and contest['gameType'] == "Pick'Em":
                    if red_eye:
                        if re.match(".*Red\sEye.*", contest['n']):
                            print (contest)
                            print ("Draft Group: " + str(contest['dg']))
                            draftGroup = contest['dg']
                            found = True
                    else:
                        print (contest)
                        print ("Draft Group: " + str(contest['dg']))
                        draftGroup = contest['dg']
                        found = True
            except:
                pass

{'uc': 0, 'ec': 0, 'mec': 1, 'fpp': 1, 's': 3, 'n': 'NHL Pick’Em $1 5-Player', 'attr': {'League': 'true'}, 'nt': 2, 'm': 5, 'a': 1, 'po': 4.5, 'pd': {'Cash': '4.5'}, 'tix': False, 'sdstring': 'Fri 7:30PM', 'sd': '/Date(1513989000000)/', 'id': 51553872, 'tmpl': 4815, 'pt': 1, 'so': 3267300, 'fwt': False, 'isOwner': False, 'startTimeType': 0, 'dg': 16966, 'ulc': 0, 'cs': 1, 'gameType': "Pick'Em", 'ssd': None, 'dgpo': 138863.4, 'cso': 3923, 'ir': 0, 'rl': False, 'rlc': 0, 'rll': 1, 'sa': True}
Draft Group: 16966


In [18]:
response = requests.get(draft_group_url + str(draftGroup) + draft_tables_suffix)

In [19]:
players = json.loads(response.text)

In [20]:
player_dict = {}
for player in players['draftables']:
    player_dict[player['firstName']+ " " + player['lastName']] = player['rosterSlotId']

In [21]:
sorted_player_dict = sorted(player_dict.items(), key=operator.itemgetter(1))

In [22]:
tmp_value = ''
string = ''
for key, value in sorted_player_dict:
    if tmp_value != str(value):
        if string != "":
            print (string[:-1] + "],")
        string = "['" + fixPlayerName(key) + "',"
    else:
        string = string + "'" + fixPlayerName(key) + "',"
    
    tmp_value = str(value)
        
print (string[:-1]+ "]")


['Vincent Trocheck','Jonathan Huberdeau','Johnny Gaudreau','Evgeny Kuznetsov'],
['Aleksander Barkov','Eric Staal','Max Pacioretty','John Carlson'],
['Jason Zucker','Evgenii Dadonov','Sean Monahan','Nicklas Backstrom'],
['Ryan Suter','Jonathan Drouin','Brendan Gallagher','Mikael Backlund','TJ Oshie','Clayton Keller'],
['Mikael Granlund','Nino Niederreiter','Nick Bjugstad','Mark Giordano','Alex Galchenyuk','Matthew Tkachuk','Oliver EkmanLarsson'],
['Aaron Ekblad','Mikko Koivu','Phillip Danault','Michael Frolik','Micheal Ferland','Derek Stepan','Tom Wilson','Jakub Vrana']
